## DB연결

In [1]:
from pymongo import MongoClient
# 네트워크 연결 규칙 : protocol://ip:port/path 
client = MongoClient('mongodb://192.168.0.50:27017/')

db = client["DB_SGMN"]
collection = db["COL_SCRAPPING_TOSS_COMMENT_HISTORY"]

In [2]:
getDatafromDB = collection.find()


data = list(getDatafromDB)


In [3]:
import pandas as pd

In [4]:
dfData = pd.DataFrame(data)

In [5]:
dfData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251836 entries, 0 to 251835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   _id         251836 non-null  object        
 1   COMMENT     251836 non-null  object        
 2   CREATED_AT  251836 non-null  datetime64[ns]
 3   DATE        251836 non-null  object        
 4   DATETIME    251836 non-null  object        
 5   SYMBOL      251836 non-null  object        
 6   UPDATED_AT  251836 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 13.4+ MB


In [6]:
dfDataEdited = dfData.drop(labels='_id',axis=1)
dfDataEdited

,COMMENT,CREATED_AT,DATE,DATETIME,SYMBOL,UPDATED_AT
0,머선일이야,2024-11-21 23:59:13.068,2024-11-22,2024-11-22T01:36:32+09:00,AADI,2024-11-21 23:58:33
1,뭐냐 너..왜 희망 갖게하냐,2024-11-21 23:59:13.068,2024-11-22,2024-11-22T01:07:42+09:00,AADI,2024-11-21 23:58:34
2,몇개월 기다리다가 조금 올라서 뺏더니 잘 올라가네.. ㅠ,2024-11-21 23:59:13.068,2024-11-22,2024-11-22T01:00:04+09:00,AADI,2024-11-21 23:58:34
3,얘는 얼마까지 갈까요,2024-11-22 00:01:24.067,2024-11-21,2024-11-21T21:44:58+09:00,AAOI,2024-11-22 00:00:13
4,"애플, 2026년 출시 목표로 시리 전면 개편 추진\n\n(주요내용)\n1. 대규모...",2024-11-22 00:01:24.067,2024-11-22,2024-11-22T07:40:12+09:00,AAPL,2024-11-22 00:00:33
...,...,...,...,...,...,...
251831,결국엔 양전 할듯.. 잘 버팀..,2024-12-23 09:24:03.263,2024-12-21,2024-12-21T00:00:57+09:00,TSLA,2024-12-23 09:24:02
251832,ㅋㅋㅋ중소주 올리고 대장들 들어올려야지?,2024-12-23 09:24:03.263,2024-12-21,2024-12-21T00:00:55+09:00,TSLA,2024-12-23 09:24:02
251833,618 가는거아니면 사면안됨,2024-12-23 09:24:03.263,2024-12-21,2024-12-21T00:00:54+09:00,TSLA,2024-12-23 09:24:02
251834,양자만 올리지말고 이놈도 같이 올려라,2024-12-23 09:24:03.263,2024-12-21,2024-12-21T00:00:33+09:00,TSLA,2024-12-23 09:24:02


In [7]:
dataToDicOnlyCommentAndDate = dfDataEdited[dfDataEdited['SYMBOL'] == 'TSLA'][['COMMENT', 'DATE', 'SYMBOL']].to_dict(orient="list")


In [8]:
commentDatasList = dfDataEdited.groupby('DATE')['COMMENT'].apply(list).to_dict()
commentDatasList
type(commentDatasList)

dict

In [9]:
dateList = dataToDicOnlyCommentAndDate['DATE']
commentList = dataToDicOnlyCommentAndDate['COMMENT']
# dateList
type(dateList)

list

In [10]:
commentList
len(commentList)

177673

## TF-IDF 사용 문장 유사도 작성

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer # Word Embedding
# tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit(commentList) # 학습용
result_vectors = tfidfVectorizer.transform(commentList) # 서비스용


In [12]:
result_vectors.toarray().shape

(177673, 86970)

In [13]:
result_vectors.toarray().shape, tfidfVectorizer.get_feature_names_out().shape

((177673, 86970), (86970,))

In [14]:
vectors_100 = result_vectors[:100]

In [15]:
## 문장간 유사도 측정

from sklearn.metrics.pairwise import cosine_similarity
df_similarity = cosine_similarity(vectors_100, vectors_100)
df_similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
import pandas as pd

pd.DataFrame(data=df_similarity

, index=commentList[:100]
, columns=commentList[:100]
)

,토스증권 업데이트땜에 더 좋아지는거 같은데\n영웅문과 mpop도 쓰는 입장에서\n토스증권은 끊을수 없을거 같다...,오를건데 뭐..,지금 tsll 사는 거 어떻게 생각하세용,1주 샀는데 혼자서 수익 4천원 이상을 버네 ㄷㄷ,왜 차이가 나는거죠,https://youtu.be/FM4uo8GBiUI?si=aUPuPBVYZskuUVM_,다시는 못살 가격...,테슬라의 턴이다,오늘 무적권이다,한국인들 전부 테슬라로 몰리는거 아냐?? 흠...,...,https://youtu.be/ykJj74uvQhE?si=5alArAnV_c3hZE_0,400 400 400 400 400 400 400 400 400 400,오늘 사면 오를까요?,https://youtu.be/SGVDmUxX8Is?si=dQJthijraUq09K1_,"오늘 모델Y 배송옵니다 ^.,^",#테슬라,"12월 4일 미 증시: 혼란 속 나스닥의 상승, 한국 계엄령과 고용 지표의 복합적 영향\n\n오늘 미 증시는 한국의 계엄령 발표라는 충격적인 외부 변수와 미국 고용 지표(JOLTs)의 견고한 결과 속에서 혼조세를 보였습니다. 개인적으로, 시장의 이런 움직임은 불확실성이 지배하는 환경에서 투자자들이 얼마나 신중하게 접근하고 있는지를 보여준다고 생각합니다.\n\n한국 계엄령 발표는 글로벌 공급망과 대외 신인도에 대한 우려를 부각시켰습니다. 삼성전자 ADR이 한때 7.5%까지 급락하고, 마이크론(+1.31%)이 반사 이익을 얻는 등 한국 반도체와 관련된 기업들의 주가 변화는 이 이슈가 얼마나 민감하게 작용했는지를 보여줍니다. 하지만 장 후반 계엄령 해제 소식이 전해지며 낙폭을 축소했고, 이는 시장이 불확실성 해소에 얼마나 민감하게 반응하는지를 다시 한번 확인시켜줬습니다.\n\n고용 지표는 견고한 고용시장을 보여줬습니다. 구인공고가 예상치를 웃돌며 고용 안정성을 시사했고, 자발적 퇴직률 상승은 근로자들의 자신감을 보여줬습니다. 하지만 채용 비율이 둔화되었다는 점은 아직 완벽한 회복으로 보기엔 부족하다고 느낍니다. 이런 데이터는 금리 인하 기대감을 높였지만, 시장은 여전히 신중한 태도를 유지하고 있습니다.\n\n결국 오늘의 시장은 불확실성과 기대감이 동시에 작용하며 혼조를 보였습니다. 개인적으로, 이런 환경에서는 종목 선택이 더 중요해질 것으로 보입니다. 특히 외부 변수에 민감하게 반응하는 업종이나 종목은 피하고, 안정적인 펀더멘털을 가진 기업에 주목할 필요가 있습니다. 고용보고서 발표가 예정된 6일까지는 이런 신중한 접근이 더 유효할 것입니다.",왜 안사가지고\n빨기->리 경제가 낳->나아져야지\n내 주식이 빚->빘을 보려나,5년안에 200만원갈종목!,꾸준매
토스증권 업데이트땜에 더 좋아지는거 같은데\n영웅문과 mpop도 쓰는 입장에서\n토스증권은 끊을수 없을거 같다...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
오를건데 뭐..,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
지금 tsll 사는 거 어떻게 생각하세용,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1주 샀는데 혼자서 수익 4천원 이상을 버네 ㄷㄷ,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
왜 차이가 나는거죠,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#테슬라,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0
"12월 4일 미 증시: 혼란 속 나스닥의 상승, 한국 계엄령과 고용 지표의 복합적 영향\n\n오늘 미 증시는 한국의 계엄령 발표라는 충격적인 외부 변수와 미국 고용 지표(JOLTs)의 견고한 결과 속에서 혼조세를 보였습니다. 개인적으로, 시장의 이런 움직임은 불확실성이 지배하는 환경에서 투자자들이 얼마나 신중하게 접근하고 있는지를 보여준다고 생각합니다.\n\n한국 계엄령 발표는 글로벌 공급망과 대외 신인도에 대한 우려를 부각시켰습니다. 삼성전자 ADR이 한때 7.5%까지 급락하고, 마이크론(+1.31%)이 반사 이익을 얻는 등 한국 반도체와 관련된 기업들의 주가 변화는 이 이슈가 얼마나 민감하게 작용했는지를 보여줍니다. 하지만 장 후반 계엄령 해제 소식이 전해지며 낙폭을 축소했고, 이는 시장이 불확실성 해소에 얼마나 민감하게 반응하는지를 다시 한번 확인시켜줬습니다.\n\n고용 지표는 견고한 고용시장을 보여줬습니다. 구인공고가 예상치를 웃돌며 고용 안정성을 시사했고, 자발적 퇴직률 상승은 근로자들의 자신감을 보여줬습니다. 하지만 채용 비율이 둔화되었다는 점은 아직 완벽한 회복으로 보기엔 부족하다고 느낍니다. 이런 데이터는 금리 인하 기대감을 높였지만, 시장은 여전히 신중한 태도를 유지하고 있습니다.\n\n결국 오늘의 시장은 불확실성과 기대감이 동시에 작용하며 혼조를 보였습니다. 개인적으로, 이런 환경에서는 종목 선택이 더 중요해질 것으로 보입니다. 특히 외부 변수에 민감하게 반응하는 업종이나 종목은 피하고, 안정적인 펀더멘털을 가진 기업에 주목할 필요가 있습니다. 고용보고서 발표가 예정된 6일까지는 이런 신중한 접근이 더 유효할 것입니다.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010726,0.0,...,0.0,0.0,0.011392,0.0,0.008822,0.0,1.0,0.0,0.0,0.0
왜 안사가지고\n빨기->리 경제가 낳->나아져야지\n내 주식이 빚->빘을 보려나,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.0
5년안에 200만원갈종목!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0


In [17]:
import pickle

# 데이터 저장하기
data_to_save = {
    'similarity_matrix': df_similarity,
    'comments': commentList[:100],  # 또는 [commentList[i] for i in random_indices]
    'vectors': vectors_100
}

# 저장
with open('similarity_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

# 나중에 불러올 때
with open('similarity_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# 불러온 데이터 사용
similarity_matrix = loaded_data['similarity_matrix']
comments = loaded_data['comments']
vectors = loaded_data['vectors']

# DataFrame 재생성
df_similarity_matrix = pd.DataFrame(
    data=similarity_matrix,
    index=comments,
    columns=comments
)
